In [66]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt


In [67]:
# Generate synthetic 2D data
np.random.seed(0)
data = np.random.multivariate_normal(mean=[0, 0], cov=[[1, 0.5], [0.5, 1]], size=1000)

# Convert data to PyTorch tensor
data_tensor = torch.tensor(data, dtype=torch.float32)


In [68]:
# Define a custom Gaussian Mixture Model (GMM) class
class GaussianMixtureModel(nn.Module):
    def __init__(self, n_components, d):
        super(GaussianMixtureModel, self).__init__()
        self.n_components = n_components
        self.d = d
        self.weights = nn.Parameter(torch.ones(n_components) / n_components)
        self.means = nn.Parameter(torch.randn(n_components, d))
        self.covars = nn.Parameter(torch.ones(n_components, d))
    
    def forward(self, x):
        x = x.unsqueeze(1)  # Add a batch dimension
        diff = x - self.means.unsqueeze(0)
        exponent = -0.5 * torch.sum(diff ** 2 / self.covars.unsqueeze(0), dim=-1)
        log_probs = exponent - 0.5 * self.d * torch.log(2 * torch.tensor(torch.pi)) - 0.5 * torch.log(self.covars.unsqueeze(0))
        log_probs = log_probs + torch.log(self.weights.unsqueeze(0))
        return torch.logsumexp(log_probs, dim=-1)
    

In [69]:
# Create the GMM model
n_components = 5  # Number of Gaussian components
d = 2  # Dimensionality of the data
gmm_model = GaussianMixtureModel(n_components, d)

# Define the optimizer
optimizer = optim.Adam(gmm_model.parameters(), lr=0.01)


In [70]:
# Training loop with score-matching objective
num_epochs = 1000
for epoch in range(num_epochs):
    gmm_log_probs = gmm_model(data_tensor)
    
    # Compute the gradient of the log-likelihood with respect to the data
    data_grad = torch.autograd.grad(gmm_log_probs.sum(), data_tensor)[0]
    
    # Compute the gradient of the model's log-likelihood for each component
    model_grad = []
    for i in range(n_components):
        component_log_probs = gmm_model(data_tensor[i])
        component_grad = torch.autograd.grad(component_log_probs.sum(), gmm_model.parameters())
        model_grad.append(component_grad)
    
    # Update the model parameters to minimize the score-matching loss
    optimizer.zero_grad()
    loss = torch.sum(data_grad ** 2)
    for i in range(n_components):
        loss -= torch.sum(model_grad[i][0] * data_grad[i])
    loss.backward()
    optimizer.step()

RuntimeError: The size of tensor a (5) must match the size of tensor b (2) at non-singleton dimension 2

In [ ]:
grid_size = 100
x_grid = np.linspace(data[:, 0].min(), data[:, 0].max(), grid_size)
y_grid = np.linspace(data[:, 1].min(), data[:, 1].max(), grid_size)
x_mesh, y_mesh = np.meshgrid(x_grid, y_grid)
grid_points = np.column_stack((x_mesh.ravel(), y_mesh.ravel()))
grid_tensor = torch.tensor(grid_points, dtype=torch.float32)
gmm_log_probs = gmm_model(grid_tensor).reshape(grid_size, grid_size).detach().numpy()


In [ ]:
plt.contourf(x_grid, y_grid, np.exp(gmm_log_probs), levels=20, cmap="viridis")
plt.colorbar()
plt.scatter(data[:, 0], data[:, 1], alpha=0.5, marker=".", color="red")
plt.xlabel("X")
plt.ylabel("Y")
plt.title("2D Gaussian Mixture Model with Score-Matching (PyTorch)")
plt.show()